## Using Fundamental Data to Predict Stock Price
### Data Retrieval and Preprocessing

**Adam Magyar and Scott Elmore** <br>
 **DSC 478**


### Description:

The dataset was put together using a combination of resources. First, we accumulated a list of the top 3000 stocks by market cap (Russell 3000).  Using these stock tickers, we used the 'yfinance' library in python to download the financial info based on quarter for each of the stocks.  Along with financial info, we downloaded the price history for as far back as we necessary in accordance to the financial info available.  This information was saved as .json files so subsequent notebooks could pull in the data by looking up the stock in a dictionary as opposed to re-downloading the data from the API, which takes some time.  During this process, some stocks were removed that didn't have all 4 quarters of necessary data.

Next, the dataset was augmented using another python library called 'ta'.  This library returns the outputs of certain technical indicators when passed in the underlying price and volume information.  We had all this information available from the yfinance API.  We decided to append 15 technical indicators that best reflect price and volume movement for each stock at the point of time that their quarterly financial info was released.  We also used another dataset that was downloaded from Bloomberg that contained the adjusted quarterly returns for each stock at each quarter.  For the regression models, we needed an appropriate target feature, so we created a column for 'Next Qtr Return' which simply looked ahead at each stocks quarterly returns and used the next quarter in the future. All this information was coalesced into one dataset 'stock_complete_info.csv'.

We decided to augment the dataset one step further by creating TFxIDF bi-grams for each stock's 'business summary'.  Each summary was around 200 words, and with around 2600 stocks, it created a very sparse matrix.  But after creating TFxIDF values for each bi-gram term and seeing the distribution of values, it was clear we could reduce this dataset to the top 500 terms with regards to TFxIDF value. This matrix was appended to the existing data and saved as 'stock_complete_info_bigrams.csv'.

Finally, the data was preprocessed to create a friendlier dataset for ML models.  Stocks varied greatly in some of their quarterly fundamental values, such as total market cap, and instead of scaling using Log, we divided each fundamental value by the number of shares each stock had outstanding.  This creates a more 'share-neutral' view of the indicators and allows for better comparison.  Some stocks didn't have values for all fundamental indicators, which we determined meant they should be 0's as opposed to dropping the row completely.  There existed some outliers even after doing a division by shares outstanding, so rows were dropped that exceeded a z-score of 3 in either direction. Finally, dummy variables were created for the 3 features that were text 'country', 'industry', and 'sector'.  This resulted in a final data set titled 'outlier_removed_processed_df_bigrams.csv'.

In [ ]:
from datetime import timedelta
import numpy as np
import pandas as pd
import yfinance as yf
import json
import ta
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt

# check https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#momentum-indicators for information on ta

### Download from YFinance API

In [ ]:
ticker_list = pd.read_csv('Russell3000Tickers.csv').values.flatten().tolist()

In [ ]:
stock_dict = {}
for ticker in ticker_list:
    stock_dict[ticker] = yf.Ticker(ticker)

In [ ]:
# create dictionaries to make dataset building process easier
historical_prices = {}
quarter_data = {}
firm_description = {}

filter_names = ['sector', 'longBusinessSummary', 'country', 'industry', 'sharesOutstanding', 'sharesShort']

for stock, obj in stock_dict.items():
    try:
        # share price & volume info
        historical_prices[stock] = obj.history(period='16mo', interval='1d')

        # quarterly info
        qb = obj.quarterly_balancesheet.T
        qf = obj.quarterly_financials.T
        qc = obj.quarterly_cashflow.T
        combined_df = pd.concat((qb, qc, qf), axis=1)
        quarter_data[stock] = combined_df

        # firm constant info
        condensed_info = { name: obj.info[name] for name in filter_names }
        firm_description[stock] = pd.DataFrame.from_records(condensed_info, index=[0])
        print('stock {} loaded'.format(stock))
    except:
        print('\n***problem loading stock : {}***\n'.format(stock))
        continue

In [ ]:
# remove stocks that don't exist for all dictionaries
remove_stocks = []
for stock_name in historical_prices.keys():
    if stock_name not in firm_description:
        remove_stocks.append(stock_name)

for stock_name in remove_stocks:
    del historical_prices[stock_name]

remove_stocks = []

for stock_name in quarter_data.keys():
    if stock_name not in firm_description:
        remove_stocks.append(stock_name)

for stock_name in remove_stocks:
    del quarter_data[stock_name]

In [ ]:
# quarter dates for price retrieval
stock_quarter_dates = {}
inadequate_stocks = []
for stock, quarter_df in quarter_data.items():
    quarter_dts = quarter_df.index.values
    quarter_dts.sort()

    # delete stock if oldest price date doesn't go back far enough
    if historical_prices[stock].index.values[0] > quarter_dts[0] + np.timedelta64(-90, 'D'):
        inadequate_stocks.append(stock)
        continue

    stock_quarter_dates[stock] = pd.DataFrame()
    for date in quarter_dts:
        find_date = True
        while find_date:
            if date in historical_prices[stock].index:
                price_dict = historical_prices[stock].loc[date]
                price_df = pd.DataFrame.from_dict(price_dict)
                prev_df = stock_quarter_dates[stock]
                stock_quarter_dates[stock] = prev_df.append(price_df.T)
                find_date = False
            else:
                date += np.timedelta64(-1, 'D')
            if date < historical_prices[stock].index.values[0]:
                inadequate_stocks.append(stock)
                find_date = False
                break

In [ ]:
for stock in inadequate_stocks:
    del quarter_data[stock]
    del historical_prices[stock]
    del firm_description[stock]

In [ ]:
# consolidate into singular df
stock_consolidated_dict = {}

for stock, quarter_price_df in stock_quarter_dates.items():
    # get columns to use
    column_list = np.concatenate((np.array(['Date']),stock_quarter_dates[stock].columns.values, quarter_data[stock].columns.values, firm_description[stock].columns.values))
    stock_df = pd.DataFrame(index=['2019_Q2', '2019_Q3', '2019_Q4', '2020_Q1'], columns=column_list)
    for i, index in enumerate(stock_df.index.values):
        append_series = pd.Series(quarter_price_df.index[i], index=['Date']).astype('str')
        append_series = append_series.append(quarter_price_df.iloc[i])
        append_series = append_series.append(quarter_data[stock].iloc[i])
        append_series = append_series.append(firm_description[stock].iloc[0])
        stock_df.loc[index] = append_series
    stock_consolidated_dict[stock] = stock_df.to_dict(orient='index')

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.datetime64):
            return str(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [ ]:
# convert dictionaries to json objects; save to directory
with open('stock_specific_info.json', 'w') as fp:
    json.dump(stock_consolidated_dict, fp, cls=NpEncoder)

In [ ]:
historical_price_dict = {}
for stock in historical_prices:
    historical_prices[stock] = historical_prices[stock].loc[~historical_prices[stock].index.duplicated(keep='last')]
    historical_price_dict[stock] = historical_prices[stock].to_dict(orient='index')
    historical_price_dict[stock] = { str(key): value for key, value in historical_price_dict[stock].items() }

In [ ]:
with open('stock_historic_prices.json', 'w') as fp:
    json.dump(historical_price_dict, fp, cls=NpEncoder)

### Augment Data with Technical Indicators

In [ ]:
# read in .json files
with open('stock_specific_info.json', 'r') as fp:
    stock_info_dict = json.load(fp)

with open('stock_historic_prices.json', 'r') as fp:
    stock_prices_dict = json.load(fp)

In [ ]:
# get quarterly returns
quarter_returns = pd.read_csv('qtrlyReturns.csv')

In [ ]:
# remove no quarter return stocks
drop_stocks = []

# add quarterly returns to stock_info_dict
for stock, quarter_info in stock_info_dict.items():
    stock_returns = quarter_returns[quarter_returns.Ticker == stock]
    try:
        stock_returns[['Q2 2019', 'Q3 2019', 'Q4 2019', 'Q1 2020', '5/21/2020']].astype('float', errors='raise')
    except:
        print('stock {} has bad qtr return values'.format(stock))
        drop_stocks.append(stock)
        continue
    stock_info_dict[stock]['2019_Q2'].update({ 'Return' : float(stock_returns['Q2 2019'].values[0] ), 'Return Next Quarter' : float(stock_returns['Q3 2019'])})
    stock_info_dict[stock]['2019_Q3'].update({ 'Return' : float(stock_returns['Q3 2019'].values[0] ), 'Return Next Quarter' : float(stock_returns['Q4 2019'])})
    stock_info_dict[stock]['2019_Q4'].update({ 'Return' : float(stock_returns['Q4 2019'].values[0] ), 'Return Next Quarter' : float(stock_returns['Q1 2020'])})
    stock_info_dict[stock]['2020_Q1'].update({ 'Return' : float(stock_returns['Q1 2020'].values[0] ), 'Return Next Quarter' : float(stock_returns['5/21/2020'])})

In [ ]:
# delete bad stocks
for stock in drop_stocks:
    del stock_info_dict[stock]
    del stock_prices_dict[stock]

In [ ]:
quarters = ['2019_Q2', '2019_Q3', '2019_Q4', '2020_Q1']

# build some techinical indicators from price history for each stock
for stock in stock_prices_dict:
    # convert to dataframe
    price_hist_df = pd.DataFrame.from_records(stock_prices_dict[stock]).T

    for qtr in quarters:
        # dictionary of indicators to add to stock info
        techinical_indicators = {}

        # momentum indicators
        aoi = ta.momentum.AwesomeOscillatorIndicator(high=price_hist_df.High, low=price_hist_df.Low).ao().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['aoi'] = aoi

        kama = ta.momentum.KAMAIndicator(close=price_hist_df.Close).kama().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['kama'] = kama

        roc = ta.momentum.ROCIndicator(close=price_hist_df.Close).roc().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['roc'] = roc

        rsi = ta.momentum.RSIIndicator(close=price_hist_df.Close).rsi().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['rsi'] = rsi

        tsi = ta.momentum.TSIIndicator(close=price_hist_df.Close).tsi().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['tsi'] = tsi

        # volume indicators
        adi = ta.volume.AccDistIndexIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close, volume=price_hist_df.Volume).acc_dist_index().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['adi'] = adi

        cmf = ta.volume.ChaikinMoneyFlowIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close, volume=price_hist_df.Volume).chaikin_money_flow().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['cmf'] = cmf

        emi = ta.volume.EaseOfMovementIndicator(high=price_hist_df.High, low=price_hist_df.Low, volume=price_hist_df.Volume).ease_of_movement().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['emi'] = emi


        # volatility indicators
        atr = ta.volatility.AverageTrueRange(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close).average_true_range().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['atr'] = atr

        bband_h_indicator = ta.volatility.BollingerBands(close=price_hist_df.Close).bollinger_hband_indicator().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['bband_h'] = bband_h_indicator

        bband_l_indicator = ta.volatility.BollingerBands(close=price_hist_df.Close).bollinger_lband_indicator().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['bband_l'] = bband_l_indicator

        # trend indicators
        adx_pos = ta.trend.ADXIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close).adx_pos().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['adx_pos'] = adx_pos

        adx_neg = ta.trend.ADXIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close).adx_neg().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['adx_neg'] = adx_neg

        macd = ta.trend.MACD(close=price_hist_df.Close).macd_signal().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['macd'] = macd

        # add to exisitng dictionary
        stock_info_dict[stock][qtr].update(techinical_indicators)


In [ ]:
# turn stock info dictionary into dataframe
stock_info_df = pd.DataFrame.from_dict({(i,j): stock_info_dict[i][j]
                           for i in stock_info_dict.keys()
                           for j in stock_info_dict[i].keys()},
                       orient='index')

In [ ]:
# combine index
stock_info_df = stock_info_df.set_index(stock_info_df.index.get_level_values(0) + ' ' + stock_info_df.index.get_level_values(1))

In [ ]:
# convert to .csv
stock_info_df.to_csv('stock_complete_info.csv')


### NLP Augmentation

In [ ]:
stock_info_df = pd.read_csv('stock_complete_info.csv', index_col=0)

In [ ]:
# do text feature extraction
# get bigrams tf_idf

company_summary_text = stock_info_df['longBusinessSummary'].values

In [ ]:
# remove special characters
def remove_string_special_characters(s):

    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', '', s)
    stripped = re.sub('_', '', stripped)

    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)

    # Remove start and end white spaces
    stripped = stripped.strip()
    if stripped != '':
            return stripped.lower()

In [ ]:
# Stopword removal and stemmer
stop_words = set(stopwords.words('english'))
overused_words = ['company', 'founded', 'inc', 'provide', 'formerly', 'known', 'offer', 'also', '']

ps = PorterStemmer()
for i, line in enumerate(company_summary_text):
    line = remove_string_special_characters(line)
    company_summary_text[i] = ' '.join([ps.stem(x) for x in word_tokenize(line) if ( x not in stop_words ) and (x not in overused_words)])


In [ ]:
# tf idf vectorizer
vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tf_idf_text = vectorizer.fit_transform(company_summary_text)
features = vectorizer.get_feature_names()


In [ ]:
# Getting top ranking bigrams
sums = tf_idf_text.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'tf_idf'])
words = (ranking.sort_values('tf_idf', ascending = True))

term_dict = dict.fromkeys(words['term'].values)
term_dict = {" ".join(sorted(key.split(" "))):term_dict[key] for key in term_dict}
term_rem_dups = list(term_dict.keys())
words = words[words['term'].isin(term_rem_dups)]
words = (ranking.sort_values('tf_idf', ascending = False))

top_500_words = words.head(500)
print ("\n\nWords : \n", top_500_words)

In [ ]:
# convert to df
tf_idf_df = pd.DataFrame(tf_idf_text.toarray(), columns = features)
del tf_idf_text

In [ ]:
# keep only top 500
tf_idf_df = tf_idf_df.loc[:, top_500_words['term'].values]

In [ ]:
# drop summary text
stock_info_df.drop(columns = 'longBusinessSummary', inplace=True)

In [ ]:
# add bigrams as features
stock_info_df = stock_info_df.reset_index().join(tf_idf_df)

In [ ]:
# set index back
stock_info_df = stock_info_df.set_index('index')

In [ ]:
# convert to .csv
stock_info_df.to_csv('stock_complete_info_bigrams.csv')

### Data Preprocessing

In [ ]:
# import csv
csv_file = 'stock_complete_info_bigrams.csv'
stock_df = pd.read_csv(csv_file, index_col=0)

In [ ]:
# drop price / volume related columns
stock_df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], inplace=True)

In [ ]:
# drop label columns
stock_df.drop(columns=['Date', 'Next Qtr', 'Ticker', 'Name'], inplace=True)

In [ ]:
# data set description
stock_df.describe()

In [ ]:
# see nan's
pd.set_option('display.max_rows', 100)
stock_df.isnull().sum()

In [ ]:
# drop columns with all nan's
stock_df.dropna(axis=1, how='all', inplace=True)

In [ ]:
# only keep rows where technical indicators exist
stock_df.dropna(axis=0, how='any', subset=['kama', 'cmf', 'emi'], inplace=True)

In [ ]:
# fill nan's with 0's for the rest
stock_df.fillna(0, inplace=True)

In [ ]:
stock_df.describe()

In [ ]:
# get location of fundamental columns
stock_df.columns.get_loc('country')
stock_df.columns.get_loc('sharesShort')
stock_df.columns.get_loc('Short Term Investments')

In [ ]:
# get fundamentals on per outstanding share basis
fundamentalsPerShare = stock_df.iloc[:,0:62].div(stock_df.sharesOutstanding, axis=0)
sharesShortPerShare = stock_df.iloc[:,67:68].div(stock_df.sharesOutstanding, axis=0)
extraFundamentalsPerShare = stock_df.iloc[:,83:85].div(stock_df.sharesOutstanding, axis=0)

In [ ]:
# transformed dataframe
stock_df.drop(columns=['sharesShort', 'Short Term Investments', 'Deferred Long Term Liab'], inplace=True)
processed_stock_df = pd.concat([fundamentalsPerShare, sharesShortPerShare, extraFundamentalsPerShare, stock_df.iloc[:,63:]], axis=1)

In [ ]:
processed_stock_df.describe()

In [ ]:
# view some data distributions
processed_stock_df.iloc[:,:19].boxplot()

In [ ]:
processed_stock_df.iloc[:,20:39].boxplot()

In [ ]:
processed_stock_df.iloc[:,40:59].boxplot()

In [ ]:
processed_stock_df.iloc[:,60:79].boxplot()

In [ ]:
from scipy import stats
outlier_removed_df = processed_stock_df[(np.abs(stats.zscore(processed_stock_df.iloc[:,:86].drop(columns=['country', 'industry', 'sector', 'Return', 'sharesOutstanding']))) < 3).all(axis=1)]

In [ ]:
# view updated data distributions
outlier_removed_df.iloc[:,:19].boxplot()

In [ ]:
outlier_removed_df.iloc[:,20:39].boxplot()

In [ ]:
outlier_removed_df.iloc[:,40:59].boxplot()

In [ ]:
outlier_removed_df.iloc[:,60:79].boxplot()

In [ ]:
outlier_removed_df.describe(include='object')

In [ ]:
# get dummies for country, industry, sector
outlier_removed_df_dummy = pd.get_dummies(outlier_removed_df, columns=['country', 'industry', 'sector'])

In [ ]:
# save this df as .csv
outlier_removed_df_dummy.to_csv('outlier_removed_processed_df_bigrams.csv')